#**PP LAB SESSION 6**

##Name: Ankita Ghosh
##Reg number: 180905354
##Sec:A Roll number:41




In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

In [ ]:
!nvcc --version

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

##Question 1

In [ ]:
%%cu
#include<stdio.h>
#include<cuda.h>
#include<stdlib.h>

__global__ void subqA(int *a,int *b,int *c,int m,int n)
{
	int id=threadIdx.x;
	if(id<m)
	{
	for(int i=0;i<n;i++)
	{
		c[id*n+i]=a[id*n+i]+b[id*n+i];
	}
	}
}

__global__ void subqB(int *a,int *b,int *c,int m,int n)
{
	int id=threadIdx.x;
	if(id<n)
	{
		for(int i=0;i<m;i++)
		{
			c[id+i*n]=a[id+i*n]+b[id+i*n];
		}
	}

}

__global__ void subqC(int *a,int *b,int *c,int m,int n)
{
	int id=threadIdx.x;
	if(id<m*n)
	{
			c[id]=a[id]+b[id];
	}

}

int main()
{
	int m=3,n=3;
	int a[3][3]={{1,2,3},{4,5,6},{7,8,9}};
	int b[3][3]={{10,11,12},{13,14,15},{16,17,18}};
	int c[m][n];
	int *d_a,*d_b,*d_c;
 
     printf("Matrix A:\n");
    for(int i=0;i<n;i++)
    {
        for(int j=0;j<m;j++)
        {
            printf("%d ",a[i][j]);
        }
        printf("\n");
    }
    printf("\n");
    
    printf("Matrix B:\n");
    for(int i=0;i<n;i++)
    {
        for(int j=0;j<m;j++)
        {
            printf("%d ",b[i][j]);
        }
        printf("\n");
    }
     printf("\n");
 
 
	int size=m*n*sizeof(int);
	cudaMalloc((void**)&d_a,size);
	cudaMalloc((void**)&d_b,size);
	cudaMalloc((void**)&d_c,size);

	cudaMemcpy(d_a,a,size,cudaMemcpyHostToDevice);
	cudaMemcpy(d_b,b,size,cudaMemcpyHostToDevice);
	
   
   printf("\na) Add row per Thread\n");
   subqA<<<1,m>>>(d_a, d_b, d_c,m,n);
   cudaMemcpy(c, d_c,size,cudaMemcpyDeviceToHost);
   for(int i=0;i<m;i++)
   {
		for(int j=0;j<n;j++)
		{
			printf("%d ",c[i][j]);
		}
		printf("\n");
   }
   
   printf("\nb)Add column per Thread\n");
   subqB<<<1,n>>>(d_a, d_b, d_c,m,n);
   cudaMemcpy(c, d_c,size,cudaMemcpyDeviceToHost);
   for(int i=0;i<m;i++)
   {
		for(int j=0;j<n;j++)
		{
			printf("%d ",c[i][j]);
		}
		printf("\n");
   }
   
   printf("\nc)Add element per Thread\n");
   subqC<<<1,m*n>>>(d_a, d_b, d_c,m,n);
   cudaMemcpy(c, d_c,size,cudaMemcpyDeviceToHost);
   for(int i=0;i<m;i++)
   {
		for(int j=0;j<n;j++)
		{
			printf("%d ",c[i][j]);
		}
		printf("\n");
   }
 
  cudaFree(d_a);
  cudaFree(d_b);
  cudaFree(d_c);
 return 0;
}

Matrix A:
1 2 3 
4 5 6 
7 8 9 

Matrix B:
10 11 12 
13 14 15 
16 17 18 


a) Add row per Thread
11 13 15 
17 19 21 
23 25 27 

b)Add column per Thread
11 13 15 
17 19 21 
23 25 27 

c)Add element per Thread
11 13 15 
17 19 21 
23 25 27 



##Question 2

In [ ]:
%%cu
#include<stdio.h>
#include<stdlib.h>

__global__ void mulMatrixa(int * A, int *B , int *C,int WA, int WB)
{
    int id =threadIdx.x;

        for(int j=0;j<WB;j++)
       {
           int sum=0;
           for(int k=0;k<WA;k++)
           {
               sum=sum+A[id*WA+k]*B[k*WB+j];
           }
           C[id*WB+j]=sum;
       }
}

__global__ void mulMatrixb(int * A, int *B , int *C,int WA, int WB,int HA)
{
    int id =threadIdx.x;
    for(int i=0;i<HA;i++)
    {
           int sum=0;
           for(int k=0;k<WA;k++)
           {
               sum=sum+A[i*WA+k]*B[k*WB+id];
           }
           C[i*WB+id]=sum;
    }
}

__global__ void mulMatrixc(int * A, int *B , int *C,int WA, int WB,int HA)
{
    int row =blockIdx.x;
    int col=threadIdx.x;
    int sum=0;
 
    for(int k=0;k<WA;k++)
    {
        sum=sum+A[row*WA+k]*B[k*WB+col];
    }
    C[row*WB+col]=sum;
}

int main()
{
    int HA=3;
    int WA=3;
 
    int HB=3;
    int WB=3;
 
    int A[HA][WA];
    int B[HB][WB];
    int C[HA][WB];
 
    for(int i=0;i<HA;i++)
    {
        for(int j=0;j<WB;j++)
        {
            A[i][j]=i+j;
        }
    }
    for(int i=0;i<HB;i++)
    {
        for(int j=0;j<WB;j++)
        {
            B[i][j]=i*j;
        }
    }
 
    if(WA!=HB)
      {
          printf("Dimenssions are not correct! ");
          return 0;
      }
 

    printf("Matrix A:\n");
    for(int i=0;i<HA;i++)
    {
        for(int j=0;j<WA;j++)
        {
            printf("%d ",A[i][j]);
        }
        printf("\n");
    }
    printf("\n");
    printf("Matrix B:\n");
    for(int i=0;i<HB;i++)
    {
        for(int j=0;j<WB;j++)
        {
            printf("%d ",B[i][j]);
        }
        printf("\n");
    }
     printf("\n");

    int sizeA=sizeof(int)*HA*WA;
    int sizeB=sizeof(int)*HB*WB;
    int sizeC=sizeof(int)*HA*WB;
    int *d_A,*d_B,*d_C;
 
    cudaMalloc((void**)&d_A,sizeA);
    cudaMalloc((void**)&d_B,sizeB);
    cudaMalloc((void**)&d_C,sizeC);
 
    cudaMemcpy(d_A,A,sizeA,cudaMemcpyHostToDevice);
    cudaMemcpy(d_B,B,sizeB,cudaMemcpyHostToDevice);

    mulMatrixa<<<1,HA>>>(d_A,d_B,d_C,WA,WB);
    cudaMemcpy(C,d_C,sizeC,cudaMemcpyDeviceToHost);
    printf("\n");
    printf("a) one thread for each row:\n");
    for(int i=0;i<HA;i++)
    {
        for(int j=0;j<WB;j++)
        {
            printf("%d ",C[i][j]);
        }
        printf("\n");
    }
 
    mulMatrixb<<<1,WB>>>(d_A,d_B,d_C,WA,WB,HA);
    cudaMemcpy(C,d_C,sizeC,cudaMemcpyDeviceToHost);
    printf("\n");
    printf("b) one thread for each column:\n");
    for(int i=0;i<HA;i++)
    {
        for(int j=0;j<WB;j++)
        {
            printf("%d ",C[i][j]);
        }
        printf("\n");
    }
 
    mulMatrixc<<<1,WB>>>(d_A,d_B,d_C,WA,WB,HA);
    cudaMemcpy(C,d_C,sizeC,cudaMemcpyDeviceToHost);
    printf("\n");
    printf("c) one thread for each element:\n");
    for(int i=0;i<HA;i++)
    {
        for(int j=0;j<WB;j++)
        {
            printf("%d ",C[i][j]);
        }
        printf("\n");
    }

    return 0;

}

Matrix A:
0 1 2 
1 2 3 
2 3 4 

Matrix B:
0 0 0 
0 1 2 
0 2 4 


a) one thread for each row:
0 5 10 
0 8 16 
0 11 22 

b) one thread for each column:
0 5 10 
0 8 16 
0 11 22 

c) one thread for each element:
0 5 10 
0 8 16 
0 11 22 

